In [1]:
import utility
from tqdm import tqdm
import os
from sklearn.cluster import KMeans
import pyvista as pv
import pandas as pd
import numpy as np
from IPython.display import Image
from collections import defaultdict
from scipy.spatial import KDTree
from scipy import stats
import math

In [2]:
import importlib
importlib.reload(utility)

<module 'utility' from '/home/simon/knorpel_v2/utility.py'>

Disable shortcuts
<script>Jupyter.keyboard_manager.disable()</script>

In [3]:
sitk_image, np_image = utility.read_image('/home/simon/Pictures/9001104/9001104_segm.mhd')

In [4]:
femoral_cartilage = utility.build_3d_cartilage_array(np_image, 3)
tibial_cartilage = utility.build_3d_cartilage_array(np_image, 4)

femoral_vectors = [list(element) for element in femoral_cartilage]
tibial_vectors = [list(element) for element in tibial_cartilage]

In [22]:
p = pv.Plotter()

x, y, z, xy = utility.get_xyz(tibial_vectors)
df = pd.DataFrame(data={'x': z, 'y': y, 'z': x}, columns=['x', 'y', 'z'])
p.add_mesh(pv.PolyData(df.to_numpy()), color='red')

x, y, z, xy = utility.get_xyz(femoral_vectors)
df = pd.DataFrame(data={'x': z, 'y': y, 'z': x}, columns=['x', 'y', 'z'])
p.add_mesh(pv.PolyData(df.to_numpy()), color='green')

tmp_vectors = utility.build_3d_cartilage_array(np_image, 1)
x, y, z, xy = utility.get_xyz(tmp_vectors)
df = pd.DataFrame(data={'x': z, 'y': y, 'z': x}, columns=['x', 'y', 'z'])
p.add_mesh(pv.PolyData(df.to_numpy()), color='blue')

tmp_vectors = utility.build_3d_cartilage_array(np_image, 2)
x, y, z, xy = utility.get_xyz(tmp_vectors)
df = pd.DataFrame(data={'x': z, 'y': y, 'z': x}, columns=['x', 'y', 'z'])
p.add_mesh(pv.PolyData(df.to_numpy()), color='yellow')

p.show_grid()
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

# Tibia

In [5]:
x, y, z, xy = utility.get_xyz(tibial_vectors)
df = pd.DataFrame(data={'x': z, 'y': y, 'z': x}, columns=['x', 'y', 'z'])

In [6]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(df.to_numpy()))
p.show_grid()
p.show()
p.render()
p.save_graphic('s1.svg')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Approach:
Construct an upper and a lower mesh out of the cartilage volume and calculate the distance between the meshes (by raytracing, nearest neighbors, etc). To this end, group by x, y coordinates and for each pair (x, y), take the maximum z coordinate for the upper mesh and the minimum z coordinate for the lower mesh.

In [7]:
max_z = df.groupby(['x', 'y']).max()
min_z = df.groupby(['x', 'y']).min()

tmp1 = [np.array(item) for item in max_z.index]
tmp2 = [item for item in max_z.to_numpy()]
max_z = np.column_stack((tmp1, tmp2))

tmp1 = [np.array(item) for item in min_z.index]
tmp2 = [item for item in min_z.to_numpy()]
min_z = np.column_stack((tmp1, tmp2))

In [8]:
upper_cloud = pv.PolyData(max_z)
lower_cloud = pv.PolyData(min_z)

In [9]:
p = pv.Plotter()
p.add_mesh(upper_cloud, color='green')
p.add_mesh(lower_cloud, color='red')
p.add_mesh(pv.PolyData(df.to_numpy()), color='blue', opacity=0.25)
p.add_legend([['bone-sided mesh', 'green'], ['cartilage-sided mesh', 'red']])
p.show_grid()
p.show()
p.render()
p.save_graphic('tibial_points.svg')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [10]:
# build polygon meshes for both point clouds using delaunay
lower_mesh = lower_cloud.delaunay_2d()
upper_mesh = upper_cloud.delaunay_2d()

We can now calculate the distance between the two meshes. In this example, we simply calculate the mean total distance using a k-d tree nearest neighbor search. For division into subregions, it is better to do raytracing from one mesh's normal vectors against the other mesh.

In [11]:
from scipy.spatial import KDTree

tree = KDTree(upper_cloud.points)
d, idx = tree.query(lower_cloud.points)
lower_cloud['distances'] = d
np.mean(d)

2.7352430942075143

In [12]:
importlib.reload(utility)

<module 'utility' from '/home/simon/knorpel_v2/utility.py'>

In [13]:
dd = defaultdict(list)
left_tibial_regions, right_tibial_regions, split_vector = utility.tibial_landmarks(lower_mesh.points)
for v in tqdm(lower_mesh.points):
    vector = np.array(v)
    # print(vector)
    label = utility.classify_tibial_point(vector[:2], left_tibial_regions, right_tibial_regions, split_vector)
    dd[label].append(vector)

dd.keys()

100%|█████████████████████████████████████| 8357/8357 [00:01<00:00, 6517.26it/s]


dict_keys(['aLT', 'iLT', 'aMT', 'eLT', 'iMT', 'eMT', 'cLT', 'cMT', 'pLT', 'pMT'])

In [14]:
left_tibial_regions

[[137, 31],
 [267, 31],
 [267, 74],
 [137, 74],
 17.0,
 array([203.42889544,  54.05171241])]

In [15]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(dd['eLT']), color='red')
p.add_mesh(pv.PolyData(dd['pLT']), color='blue')
p.add_mesh(pv.PolyData(dd['cLT']), color='green')
p.add_mesh(pv.PolyData(dd['aLT']), color='pink')
p.add_mesh(pv.PolyData(dd['iLT']), color='yellow')
p.add_mesh(pv.PolyData(dd['eMT']), color='red')
p.add_mesh(pv.PolyData(dd['pMT']), color='blue')
p.add_mesh(pv.PolyData(dd['cMT']), color='green')
p.add_mesh(pv.PolyData(dd['aMT']), color='pink')
p.add_mesh(pv.PolyData(dd['iMT']), color='yellow')
p.add_legend([['external', 'red'], ['posterior', 'blue'], ['anterior', 'pink'], ['internal', 'yellow'], ['central', 'green']])
p.add_title('Subregions of the tibial cartilage')
p.show_grid()
p.show()
p.render()
p.save_graphic('tibial_subregions.svg')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

# Femur

In [7]:
x, y, z, xy = utility.get_xyz(femoral_vectors)
df = pd.DataFrame(data={'x': z, 'y': y, 'z': x}, columns=['x', 'y', 'z'])

In [17]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(df.to_numpy()))
p.show_grid()
p.show()
p.render()
p.save_graphic('s2.svg')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Approach:
Using the same approach as for the tibial cartilage is not possible due to the volume's shape. Let's illustrate the problem: <br><br>
Instead of going by z coordinate, it makes more sense to take a central point and introduce a distance measure to that point.

In [37]:
center = np.array([df.x.min() + (df.x.max() - df.x.min()) / 2,
                   df.y.min() + (df.y.max() - df.y.min()) / 2,
                   df.z.min() + (df.z.max() - df.z.min()) / 2])

df['dist_to_cog'] = np.zeros(df.shape[0])
df['dist_to_cog'] = df.apply(lambda l: utility.vector_distance([l.x, l.y, l.z], center), axis=1)
df

,x,y,z,dist_to_cog
0,170,0,52,93.364876
1,171,0,52,94.180677
2,169,0,53,92.504054
3,170,0,53,93.316665
4,171,0,53,94.132885
...,...,...,...,...
122282,139,108,99,81.884064
122283,140,108,99,82.437855
122284,141,108,99,83.000000
122285,142,108,99,83.570330


If we now again group by x, y and take the min/max distance...

In [38]:
max_z = df[df.groupby(['x', 'y'])['dist_to_cog'].transform(max) == df['dist_to_cog']]
min_z = df[df.groupby(['x', 'y'])['dist_to_cog'].transform(min) == df['dist_to_cog']]

max_z = [item[:3] for item in max_z.to_numpy()]
min_z = [item[:3] for item in min_z.to_numpy()]

In [39]:
upper_cloud = pv.PolyData(max_z)
lower_cloud = pv.PolyData(min_z)

In [40]:
p = pv.Plotter()
p.add_mesh(upper_cloud, color='red')
p.add_mesh(lower_cloud, color='green')
p.add_mesh(pv.PolyData(df.to_numpy()[:,0:3]), color='blue', opacity=0.25)
p.add_legend([['bone-sided mesh', 'green'], ['cartilage-sided mesh', 'red']])
p.show_grid()
p.show()
p.render()
p.save_graphic('femoral_points.svg')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

This is what we get. You can see that points are missing in the curved regions. This makes sense because we only take one value each for each (x, y) while for these regions, we'd need >= 2 values for the inner and outer mesh.<br><br>
Another way to approach this problem is again taking a central point, and from this point do raytracing against the cartilage volume. For each tracing vector, find the first and last intersection point with the volume (if any) and calculate the distance between the two.<br><br>
To this end, we can build a sphere and trace along the sphere's normal vectors.

In [41]:
num_sp = 30
sphere = pv.Sphere(center=center, radius=1, theta_resolution=num_sp, phi_resolution=num_sp)
sphere.compute_normals(point_normals=True, cell_normals=False, inplace=True)

PolyData (0x7f41a8b8fe20)
  N Cells:	1680
  N Points:	842
  X Bounds:	9.300e+01, 9.500e+01
  Y Bounds:	5.301e+01, 5.499e+01
  Z Bounds:	5.600e+01, 5.800e+01
  N Arrays:	1

In [42]:
p = pv.Plotter()
m = pv.PolyData(df.to_numpy()[:,0:3])
p.add_mesh(m, color='red')
p.add_mesh(sphere, color='green')
# p.add_mesh(sphere.glyph(scale='Normals', orient='Normals', tolerance=0.05), color='blue')
p.add_arrows(sphere.points, sphere['Normals'], 25)
p.show_grid()
p.add_title('Sphere placement for the femoral cartilage')
p.show()
p.render()
p.save_graphic('femoral_sphere.svg')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Fixing the mesh approach
Another approach that might work is cutting the full volume into two shapes and rotating the problematic section. Let's have a look:

In [43]:
df = df.sort_values(by='x', ascending=True)
zrange = df.groupby(by=['x'])['z'].max() - df.groupby(by=['x'])['z'].min()
zrange.describe()

count    189.000000
mean      47.878307
std       30.161286
min       13.000000
25%       28.000000
50%       32.000000
75%       58.000000
max      112.000000
Name: z, dtype: float64

In [44]:
zmed = zrange.median()

In [45]:
from scipy import stats

In [46]:
zindex = zrange.loc[zrange < zmed].index.to_numpy()
mask = np.abs(stats.zscore(zindex)) < 2
lower_bound = zindex[mask].min()
upper_bound = zindex[mask].max()

In [47]:
#lower_bound = 35
#upper_bound = 122
print(lower_bound, upper_bound)

35 122


In [48]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(df.loc[df['x'] > lower_bound].loc[df['x'] < upper_bound].to_numpy()[:,0:3]), color='blue', opacity=0.25)
p.add_mesh(pv.PolyData(df.loc[df['x'] < lower_bound].to_numpy()[:,0:3]), color='red', opacity=0.25)
p.add_mesh(pv.PolyData(df.loc[df['x'] > upper_bound].to_numpy()[:,0:3]), color='yellow', opacity=0.25)
p.show()
p.render()
p.save_graphic('s3.svg')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [49]:
right_portion = df.loc[df['x'] < lower_bound]
right_portion = right_portion[['z', 'y', 'x', 'dist_to_cog']]
right_portion.columns = ['x', 'y', 'z', 'dist_to_cog']

middle_portion = df.loc[df['x'] > lower_bound].loc[df['x'] < upper_bound]

left_portion = df.loc[df['x'] > upper_bound]
left_portion = left_portion[['z', 'y', 'x', 'dist_to_cog']]
left_portion.columns = ['x', 'y', 'z', 'dist_to_cog']

In [50]:
pv.plot(pv.PolyData(left_portion.to_numpy()[:,0:3]), color='yellow')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [51]:
pv.plot(pv.PolyData(right_portion.to_numpy()[:,0:3]), color='red')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [52]:
max_z = left_portion[left_portion.groupby(['x', 'y'])['dist_to_cog'].transform(max) == left_portion['dist_to_cog']]
min_z = left_portion[left_portion.groupby(['x', 'y'])['dist_to_cog'].transform(min) == left_portion['dist_to_cog']]

max_z = [item[:3] for item in max_z.to_numpy()]
min_z = [item[:3] for item in min_z.to_numpy()]

In [53]:
outer_cloud = pv.PolyData(max_z)
inner_cloud = pv.PolyData(min_z)

In [54]:
p = pv.Plotter()
p.add_mesh(inner_cloud, color='red', opacity=.5)
p.add_mesh(outer_cloud, color='blue', opacity=.5)
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [55]:
outer_delaunay = outer_cloud.delaunay_2d()
inner_delaunay = inner_cloud.delaunay_2d()

In [56]:
p = pv.Plotter()
p.add_mesh(inner_delaunay, color='red', opacity=.5)
p.add_mesh(outer_delaunay, color='blue', opacity=.5)
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

This seems to work all right. Let's do some raytracing.

In [57]:
def build_delaunay(portion):
    max_dist = portion[portion.groupby(['x', 'y'])['dist_to_cog'].transform(max) == portion['dist_to_cog']]
    min_dist = portion[portion.groupby(['x', 'y'])['dist_to_cog'].transform(min) == portion['dist_to_cog']]

    max_dist = [item[:3] for item in max_dist.to_numpy()]
    min_dist = [item[:3] for item in min_dist.to_numpy()]
    
    outer_cloud = pv.PolyData(max_dist)
    inner_cloud = pv.PolyData(min_dist)
    
    return outer_cloud.delaunay_2d(), inner_cloud.delaunay_2d()

In [58]:
left_outer, left_inner = build_delaunay(left_portion)
middle_outer, middle_inner = build_delaunay(middle_portion)
right_outer, right_inner = build_delaunay(right_portion)

In [59]:
left_out = left_outer.copy()
left_out.points = np.array([[x[2], x[1], x[0]] for x in left_out.points])
left_in = left_inner.copy()
left_in.points = np.array([[x[2], x[1], x[0]] for x in left_in.points])

middle_out = middle_outer.copy()
middle_out.points = np.array([[x[0], x[1], x[2]] for x in middle_out.points])
middle_in = middle_inner.copy()
middle_in.points = np.array([[x[0], x[1], x[2]] for x in middle_in.points])

right_out = right_outer.copy()
right_out.points = np.array([[x[2], x[1], x[0]] for x in right_out.points])
right_in = right_inner.copy()
right_in.points = np.array([[x[2], x[1], x[0]] for x in right_in.points])

In [60]:
p = pv.Plotter()
p.add_mesh(left_out, color='red')
p.add_mesh(middle_out, color='red')
p.add_mesh(right_out, color='red')
p.add_mesh(left_in, color='green')
p.add_mesh(middle_in, color='green')
p.add_mesh(right_in, color='green')
p.show_grid()
p.add_legend([['bone-sided mesh', 'green'], ['cartilage-sided mesh', 'red']])
p.add_title('Succesful construction of meshes for the femoral cartilage', font_size=10)

p.show()
p.render()
p.save_graphic('femoral_meshes.svg')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [61]:
cluster = KMeans(n_clusters=1, random_state=0).fit(femoral_vectors)
split_vector = cluster.cluster_centers_[0]
femoral_split_vector = split_vector
left_plate, right_plate = utility.split_into_plates(femoral_vectors, split_vector)

first_split, second_split = utility.get_femoral_thirds(left_plate)
left_femoral_regions = [first_split, second_split]

first_split, second_split = utility.get_femoral_thirds(right_plate)
right_femoral_regions = [first_split, second_split]

In [62]:
def ray_trace(inner, outer):
    average_thickness = dict()
    n_points = inner.n_points
    average_thickness['ecLF'] = np.zeros(n_points)
    average_thickness['ccLF'] = np.zeros(n_points)
    average_thickness['icLF'] = np.zeros(n_points)
    average_thickness['icMF'] = np.zeros(n_points)
    average_thickness['ccMF'] = np.zeros(n_points)
    average_thickness['ecMF'] = np.zeros(n_points)
    
    inner_normals = inner.compute_normals(cell_normals=False)
    inner_normals['distances'] = np.zeros(inner.n_points)
    for i in range(inner_normals.n_points):
        v = inner.points[i]
        vec = inner_normals['Normals'][i] * inner_normals.length
        v0 = v - vec
        v1 = v + vec
        iv, ic = outer.ray_trace(v0, v1, first_point=True)
        dist = np.sqrt(np.sum((iv - v)**2))
        inner_normals['distances'][i] = dist
        label = utility.classify_femoral_point(v[:2], left_femoral_regions, right_femoral_regions, split_vector)
        average_thickness[label][i] = dist
    
    return average_thickness

In [63]:
left_thickness = ray_trace(left_inner, left_outer)
middle_thickness = ray_trace(middle_inner, middle_outer)
right_thickness = ray_trace(right_inner, right_outer)

In [64]:
average_thickness = {key: np.hstack((left_thickness[key], middle_thickness[key], right_thickness[key])) for key in left_thickness.keys()}

In [65]:
for key, value in average_thickness.items():
    mask = value == 0
    value[mask] = np.nan
    print(f'{key}: {np.nanmean(value)}')

ecLF: 4.129961834403946
ccLF: 5.005532149065432
icLF: 5.453199175673108
icMF: 6.514685954736944
ccMF: 4.35446237196326
ecMF: 3.9881449550694197


In [66]:
from collections import defaultdict


In [67]:
importlib.reload(utility)

<module 'utility' from '/home/simon/knorpel_v2/utility.py'>

In [68]:
df.to_numpy()[:,:3]

array([[  0.,  88.,  44.],
       [  0.,  90.,  48.],
       [  0.,  89.,  45.],
       ...,
       [188.,  18.,  54.],
       [188.,  23.,  53.],
       [188.,  13.,  38.]])

In [69]:
dd = defaultdict(list)
for vector in tqdm(df.to_numpy()[:,:3]):
    label = utility.classify_femoral_point(vector[:2], left_femoral_regions, right_femoral_regions, split_vector)
    dd[label].append(vector)

100%|███████████████████████████████| 122287/122287 [00:00<00:00, 307646.13it/s]


In [70]:
dd.keys()

dict_keys(['ccMF', 'ecMF', 'icMF', 'icLF', 'ccLF', 'ecLF'])

In [71]:
print(left_femoral_regions, right_femoral_regions, split_vector)

[22, 34] [77, 88] [ 75.94043521  55.95815581 110.1472356 ]


In [72]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(dd['ecLF']), color='red')
p.add_mesh(pv.PolyData(dd['icLF']), color='yellow')
p.add_mesh(pv.PolyData(dd['ccLF']), color='green')
p.add_mesh(pv.PolyData(dd['icMF']), color='pink')
p.add_mesh(pv.PolyData(dd['ecMF']), color='red')
p.add_mesh(pv.PolyData(dd['ccMF']), color='green')
p.show_grid()
p.add_legend([['external', 'red'], ['central', 'green'], ['internal (left)', 'yellow'], ['internal (right)', 'pink']])
p.add_title('Subregions of the femoral cartilage')
p.show()
p.render()
p.save_graphic('femoral_subregions.svg')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Combine outer clouds for the regional classification

In [73]:
ldf = pd.DataFrame(data=left_outer.points, columns=['x', 'y', 'z'])
mdf = pd.DataFrame(data=middle_outer.points, columns=['x', 'y', 'z'])
rdf = pd.DataFrame(data=right_outer.points, columns=['x', 'y', 'z'])

ldf = ldf[['z', 'y', 'x']]
ldf.columns = ['x', 'y', 'z']

rdf = rdf[['z', 'y', 'x']]
rdf.columns = ['x', 'y', 'z']

cdf = pd.concat([ldf, mdf, rdf])

In [74]:
# outer_stack = np.vstack((left_outer.points, middle_outer.points, right_outer.points))
pv.PolyData(cdf.to_numpy()).plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Sphere approach for the tibial cartilage
Because why not?

In [80]:
x, y, z, xy = utility.get_xyz(tibial_vectors)
df = pd.DataFrame(data={'x': z, 'y': y, 'z': x}, columns=['x', 'y', 'z'])

In [81]:
center = np.array([df.x.min() + (df.x.max() - df.x.min()) / 2,
                   df.y.min() + (df.y.max() - df.y.min()) / 2,
                   df.z.max() * 1.25])
                   #df.z.min() + (df.z.max() - df.z.min()) / 2])

df['dist_to_cog'] = np.zeros(df.shape[0])
df['dist_to_cog'] = df.apply(lambda l: utility.vector_distance([l.x, l.y, l.z], center), axis=1)
df

,x,y,z,dist_to_cog
0,76,0,23,57.768936
1,77,0,23,57.976288
2,78,0,23,58.200086
3,79,0,23,58.440140
4,62,0,24,56.375970
...,...,...,...,...
35793,68,108,23,56.720807
35794,69,108,23,56.791285
35795,70,108,23,56.879258
35796,71,108,23,56.984647


In [82]:
num_sp = 30
sphere = pv.Sphere(center=center, radius=1, theta_resolution=num_sp, phi_resolution=num_sp)
sphere.compute_normals(point_normals=True, cell_normals=False, inplace=True)

PolyData (0x7fb82466ee20)
  N Cells:	1680
  N Points:	842
  X Bounds:	6.350e+01, 6.550e+01
  Y Bounds:	5.301e+01, 5.499e+01
  Z Bounds:	3.900e+01, 4.100e+01
  N Arrays:	1

In [86]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(df.to_numpy()[:,0:3]), color='red')
p.add_mesh(sphere, color='green')
# p.add_mesh(sphere.glyph(scale='Normals', orient='Normals', tolerance=0.05), color='blue')
p.add_arrows(sphere.points, sphere['Normals'], 5)
p.show_grid()
p.add_title('Sphere placement for the tibial cartilage')
p.show()
p.render()
p.save_graphic('tibial_sphere.svg')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Extraction of the weight-bearing zone of the femoral cartilage

In [4]:
femoral_cartilage_new = [0] * (np_image.shape[0] * np_image.shape[1] * np_image.shape[2])
tibial_cartilage_new = [0] * (np_image.shape[0] * np_image.shape[1] * np_image.shape[2])
tindx = 0
findx = 0
for y in tqdm(range(np_image.shape[0])):
    for x in range(np_image.shape[1]):
        for z in range(np_image.shape[2]):
            if np_image[y,x,z] == 3:
                femoral_cartilage_new[findx] = [x, y, z]
                findx += 1
            if np_image[y,x,z] == 4:
                tibial_cartilage_new[tindx] = [x, y, z]
                tindx += 1

femoral_cartilage_new = np.array(femoral_cartilage_new, dtype=object)
femoral_cartilage_new = femoral_cartilage_new[femoral_cartilage_new != 0] 
femoral_cartilage_new = [list(element) for element in femoral_cartilage_new]

tibial_cartilage_new = np.array(tibial_cartilage_new, dtype=object)
tibial_cartilage_new = tibial_cartilage_new[tibial_cartilage_new != 0] 
tibial_cartilage_new = [list(element) for element in tibial_cartilage_new]

100%|█████████████████████████████████████████| 160/160 [02:38<00:00,  1.01it/s]


In [5]:
x, y, z, xy = utility.get_xyz(tibial_cartilage_new)
df = pd.DataFrame(data={'x': z, 'y': y, 'z': x}, columns=['x', 'y', 'z'])
max_z = df.groupby(['x', 'y']).max()
min_z = df.groupby(['x', 'y']).min()

tmp1 = [np.array(item) for item in max_z.index]
tmp2 = [item for item in max_z.to_numpy()]
max_z = np.column_stack((tmp1, tmp2))

tmp1 = [np.array(item) for item in min_z.index]
tmp2 = [item for item in min_z.to_numpy()]
min_z = np.column_stack((tmp1, tmp2))

upper_cloud = pv.PolyData(max_z)

dd = defaultdict(list)
left_tibial_regions, right_tibial_regions, split_vector = utility.tibial_landmarks(max_z)
for v in tqdm(max_z):
    vector = np.array(v)
    # print(vector)
    label = utility.classify_tibial_point(vector[:2], left_tibial_regions, right_tibial_regions, split_vector)
    dd[label].append(vector)

dd.keys()

100%|█████████████████████████████████████| 8357/8357 [00:01<00:00, 7551.71it/s]


dict_keys(['aLT', 'iLT', 'aMT', 'eLT', 'iMT', 'eMT', 'cLT', 'cMT', 'pLT', 'pMT'])

In [6]:
wbl = np.vstack((np.array(dd['eLT']), np.array(dd['cLT']), np.array(dd['iLT'])))
wbr = np.vstack((np.array(dd['iMT']), np.array(dd['cMT']), np.array(dd['eMT'])))

In [7]:
tdfr = pd.DataFrame(data={'x': wbr[:,0], 'y': wbr[:,1], 'z': wbr[:,2]})
tdfl = pd.DataFrame(data={'x': wbl[:,0], 'y': wbl[:,1], 'z': wbl[:,2]})

In [8]:
print(max(np.array(dd['cMT'])[:,0]), max(np.array(dd['eMT'])[:,0]), max(np.array(dd['iMT'])[:,0]))

231 253 255


In [9]:
tdfr = tdfr.loc[tdfr['x'] < max(np.array(dd['cMT'])[:,0])].loc[tdfr['x'] > min(np.array(dd['cMT'])[:,0])]
tdfl = tdfl.loc[tdfl['x'] < max(np.array(dd['cLT'])[:,0])].loc[tdfl['x'] > min(np.array(dd['cLT'])[:,0])]

In [10]:
x, y, z, xy = utility.get_xyz(femoral_cartilage_new)
df = pd.DataFrame(data={'x': z, 'y': y, 'z': x}, columns=['x', 'y', 'z'])
fdfl = df.loc[df['y'] < df['y'].mean()]
fdfr = df.loc[df['y'] >= df['y'].mean()]

In [11]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(fdfl.to_numpy()), color='blue')
p.add_mesh(pv.PolyData(fdfr.to_numpy()), color='purple')
p.add_mesh(pv.PolyData(tdfl.to_numpy()), color='green')
p.add_mesh(pv.PolyData(tdfr.to_numpy()), color='yellow')
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [12]:
tree = KDTree(tdfl.to_numpy())
d, idx = tree.query(fdfl.to_numpy())
d

array([75.00666637, 75.45197148, 73.89181281, ..., 25.41653005,
       24.55605832, 23.70653918])

In [13]:
# wbz = np.array(femoral_cartilage_new)[np.where(d < 10)]
wbz = np.array(fdfl.to_numpy()[np.where(d < 10)])

In [14]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(fdfl.to_numpy()), color='blue')
p.add_mesh(pv.PolyData(tdfl.to_numpy()), color='green')
p.add_mesh(pv.PolyData(wbz), color='red')
#p.show_grid()
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [17]:
minx = tdfl['x'].min()
maxx = tdfl['x'].max()
miny = tdfl['y'].min()
maxy = tdfl['y'].max()
wbz = fdfl.loc[fdfl['x'] > minx].loc[fdfl['x'] < maxx].loc[fdfl['y'] > miny].loc[fdfl['y'] < maxy].to_numpy()

In [19]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(fdfl.to_numpy()), color='blue')
p.add_mesh(pv.PolyData(tdfl.to_numpy()), color='green')
p.add_mesh(pv.PolyData(fdfr.to_numpy()), color='orange')
p.add_mesh(pv.PolyData(wbz), color='red')
#p.show_grid()
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

# New approach for the femur

In [81]:
importlib.reload(utility)

<module 'utility' from '/home/simon/knorpel_v2/utility.py'>

In [5]:
cwbzl, cwbzr = utility.extract_central_weightbearing_zone(femoral_vectors, tibial_vectors)

In [8]:
x, y, z, xy = utility.get_xyz(femoral_vectors)
df = pd.DataFrame(data={'x': z, 'y': y, 'z': x}, columns=['x', 'y', 'z'])
p = pv.Plotter()
p.add_mesh(pv.PolyData(df.to_numpy()))
p.add_mesh(pv.PolyData(cwbzl.to_numpy()), color='red')
p.add_mesh(pv.PolyData(cwbzr.to_numpy()), color='green')
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [9]:
lower_mesh_left, upper_mesh_left = utility.build_tibial_meshes(cwbzl.to_numpy())
lower_mesh_right, upper_mesh_right = utility.build_tibial_meshes(cwbzr.to_numpy())

In [45]:
p = pv.Plotter()
p.add_mesh(lower_mesh_left, color='red')
p.add_mesh(upper_mesh_left, color='green')
p.add_mesh(lower_mesh_right, color='blue')
p.add_mesh(upper_mesh_right, color='purple')
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [11]:
left_landmarks = utility.femoral_landmarks(upper_mesh_left.points)
right_landmarks = utility.femoral_landmarks(upper_mesh_right.points)

In [12]:
left_left_portion = cwbzl.loc[cwbzl['y'] < left_landmarks[0]]
left_middle_portion = cwbzl.loc[cwbzl['y'] >= left_landmarks[0]].loc[cwbzl['y'] < left_landmarks[1]]
left_right_portion = cwbzl.loc[cwbzl['y'] >= left_landmarks[1]]

In [13]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(left_left_portion.to_numpy()), color='blue')
p.add_mesh(pv.PolyData(left_middle_portion.to_numpy()), color='red')
p.add_mesh(pv.PolyData(left_right_portion.to_numpy()), color='green')
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [14]:
left_right_portion = left_right_portion[['z', 'y', 'x']]
left_right_portion.columns = ['x', 'y', 'z']

In [15]:
left_portion_lower_mesh, left_portion_upper_mesh = utility.build_tibial_meshes(left_left_portion.to_numpy())
middle_portion_lower_mesh, middle_portion_upper_mesh = utility.build_tibial_meshes(left_middle_portion.to_numpy())
right_portion_lower_mesh, right_portion_upper_mesh = utility.build_tibial_meshes(left_right_portion.to_numpy())

In [16]:
p = pv.Plotter()
p.add_mesh(left_portion_lower_mesh, color='blue')
p.add_mesh(middle_portion_lower_mesh, color='red')
p.add_mesh(right_portion_lower_mesh, color='green')
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [17]:
dd = defaultdict(list)
for vector in tqdm(cwbzl.to_numpy()[:,:3]):
    label = utility.classify_femoral_point(vector[:2], left_landmarks, True)
    dd[label].append(vector)

100%|███████████████████████████████████| 5016/5016 [00:00<00:00, 344162.09it/s]


In [18]:
dd.keys()

dict_keys(['ecLF', 'ccLF', 'icLF'])

In [19]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(dd['ecLF']), color='red')
p.add_mesh(pv.PolyData(dd['icLF']), color='yellow')
p.add_mesh(pv.PolyData(dd['ccLF']), color='green')
p.show_grid()
p.add_legend([['external', 'red'], ['central', 'green'], ['internal (left)', 'yellow'], ['internal (right)', 'pink']])
p.add_title('Subregions of the femoral cartilage')
p.show()
# p.render()
# p.save_graphic('femoral_subregions.svg')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [20]:
dd = defaultdict(list)
for vector in tqdm(cwbzr.to_numpy()[:,:3]):
    label = utility.classify_femoral_point(vector[:2], right_landmarks, False)
    dd[label].append(vector)

100%|███████████████████████████████████| 5815/5815 [00:00<00:00, 403485.27it/s]


In [21]:
dd.keys()

dict_keys(['icMF', 'ccMF', 'ecMF'])

In [22]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(dd['icMF']), color='pink')
p.add_mesh(pv.PolyData(dd['ecMF']), color='red')
p.add_mesh(pv.PolyData(dd['ccMF']), color='green')
p.show_grid()
p.add_legend([['external', 'red'], ['central', 'green'], ['internal (left)', 'yellow'], ['internal (right)', 'pink']])
p.add_title('Subregions of the femoral cartilage')
p.show()
# p.render()
# p.save_graphic('femoral_subregions.svg')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [23]:
left_thickness = dict()
left_thickness['ecLF'] = np.zeros(lower_mesh_left.n_points)
left_thickness['ccLF'] = np.zeros(lower_mesh_left.n_points)
left_thickness['icLF'] = np.zeros(lower_mesh_left.n_points)

right_thickness = dict()
right_thickness['icMF'] = np.zeros(lower_mesh_right.n_points)
right_thickness['ccMF'] = np.zeros(lower_mesh_right.n_points)
right_thickness['ecMF'] = np.zeros(lower_mesh_right.n_points)

left_normals = lower_mesh_left.compute_normals(cell_normals=False)
right_normals = lower_mesh_right.compute_normals(cell_normals=False)

_, left_thickness = utility.calculate_femoral_thickness(left_normals, lower_mesh_left, upper_mesh_left, sitk_image, left_landmarks, left_thickness, True)
_, right_thickness = utility.calculate_femoral_thickness(right_normals, lower_mesh_right, upper_mesh_right, sitk_image, right_landmarks, right_thickness, False)

In [24]:
femoral_thickness = dict()
femoral_thickness.update(left_thickness)
femoral_thickness.update(right_thickness)

In [25]:
femoral_thickness

{'ecLF': array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.      

In [28]:
keys = set(femoral_thickness.keys())
for key in keys:
    value = femoral_thickness[key]
    mask = value == 0
    value[mask] = np.nan
    femoral_thickness[key + '.aSD'] = np.nanstd(value)
    femoral_thickness[key + '.aMav'] = np.nanmean(-np.sort(-value)[:math.ceil(len(value) * 0.01)])
    femoral_thickness[key + '.aMiv'] = np.nanmean(np.sort(value)[:math.ceil(len(value) * 0.01)])
    femoral_thickness[key] = np.nanmean(value)

In [29]:
femoral_thickness

{'ecLF': 5.520913356509978,
 'ccLF': 5.7709171071388905,
 'icLF': 3.019644067941344,
 'icMF': 5.683730386914962,
 'ccMF': 7.372331942134219,
 'ecMF': 8.933682570047049,
 'ecLF.aSD': 5.470576015636847,
 'ecLF.aMav': 22.660984930927476,
 'ecLF.aMiv': 1.2699833486323233,
 'ccLF.aSD': 5.030968934821199,
 'ccLF.aMav': 18.7599464,
 'ccLF.aMiv': 1.691904296009784,
 'icLF.aSD': 2.4140239593103625,
 'icLF.aMav': 11.7599664,
 'icLF.aMiv': 0.4329339489669343,
 'ccMF.aSD': 5.526537363239277,
 'ccMF.aMav': 18.899946,
 'ccMF.aMiv': 2.1218393462359715,
 'icMF.aSD': 4.178854694744948,
 'icMF.aMav': 15.399956000000001,
 'icMF.aMiv': 1.834222664380698,
 'ecMF.aSD': 6.541115694776559,
 'ecMF.aMav': 20.33587553169133,
 'ecMF.aMiv': 1.8467537010987567}

In [30]:
importlib.reload(utility)

<module 'utility' from '/home/simon/knorpel_v2/utility.py'>

In [31]:
ladf, radf, pdf = utility.extract_anterior_posterior_zones(femoral_vectors, cwbzl, cwbzr)

In [32]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(ladf.to_numpy()), color='red')
p.add_mesh(pv.PolyData(radf.to_numpy()), color='green')
p.add_mesh(pv.PolyData(pdf.to_numpy()), color='blue')
# p.show_grid()
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [33]:
p = pv.Plotter()
p.add_mesh(pv.PolyData(ladf.to_numpy()), color='red')
p.add_mesh(pv.PolyData(radf.to_numpy()), color='green')
p.show_grid()
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [34]:
la_lower_mesh, la_upper_mesh = utility.build_tibial_meshes(ladf.to_numpy())
ra_lower_mesh, ra_upper_mesh = utility.build_tibial_meshes(radf.to_numpy())
p_lower_mesh, p_upper_mesh = utility.build_tibial_meshes(pdf.to_numpy())

In [35]:
p = pv.Plotter()
p.add_mesh(la_lower_mesh, color='red')
p.add_mesh(la_upper_mesh, color='orange')
p.add_mesh(ra_lower_mesh, color='green')
p.add_mesh(ra_upper_mesh, color='cyan')
p.show_grid()
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [36]:
p = pv.Plotter()
p.add_mesh(p_lower_mesh, color='red')
p.add_mesh(p_upper_mesh, color='green')
p.show_grid()
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [37]:
la_normals = la_lower_mesh.compute_normals(cell_normals=False)
ra_normals = ra_lower_mesh.compute_normals(cell_normals=False)
p_normals = p_lower_mesh.compute_normals(cell_normals=False)

la_distances = utility.calculate_distance_without_classification(la_normals, la_lower_mesh, la_upper_mesh, sitk_image)
ra_distances = utility.calculate_distance_without_classification(ra_normals, ra_lower_mesh, ra_upper_mesh, sitk_image)
p_distances = utility.calculate_distance_without_classification(p_normals, p_lower_mesh, p_upper_mesh, sitk_image)

In [38]:
la_distances['distances']

array([0.      , 0.      , 0.      , ..., 0.699998, 0.699998, 0.699998])

In [39]:
la_thickness = dict()
distances = la_distances['distances']
mask = distances == 0
distances[mask] = np.nan
la_thickness['aLF.aSD'] = np.nanstd(distances)
la_thickness['aLF.aMav'] = np.nanmean(-np.sort(-distances)[:math.ceil(len(distances) * 0.01)])
la_thickness['aLF.aMiv'] = np.nanmean(np.sort(distances)[:math.ceil(len(distances) * 0.01)])
la_thickness['aLF'] = np.nanmean(distances)

In [40]:
la_thickness

{'aLF.aSD': 3.682084079496431,
 'aLF.aMav': 29.62488843684523,
 'aLF.aMiv': 0.4799104450079946,
 'aLF': 3.4902455918770516}

In [41]:
ra_thickness = dict()
distances = ra_distances['distances']
mask = distances == 0
distances[mask] = np.nan
ra_thickness['aMF.aSD'] = np.nanstd(distances)
ra_thickness['aMF.aMav'] = np.nanmean(-np.sort(-distances)[:math.ceil(len(distances) * 0.01)])
ra_thickness['aMF.aMiv'] = np.nanmean(np.sort(distances)[:math.ceil(len(distances) * 0.01)])
ra_thickness['aMF'] = np.nanmean(distances)

In [42]:
ra_thickness

{'aMF.aSD': 2.353725752024288,
 'aMF.aMav': 17.74147305280583,
 'aMF.aMiv': 0.6275746202189278,
 'aMF': 3.0626394904927174}

In [43]:
p_thickness = dict()
distances = p_distances['distances']
mask = distances == 0
distances[mask] = np.nan
p_thickness['pF.aSD'] = np.nanstd(distances)
p_thickness['pF.aMav'] = np.nanmean(-np.sort(-distances)[:math.ceil(len(distances) * 0.01)])
p_thickness['pF.aMiv'] = np.nanmean(np.sort(distances)[:math.ceil(len(distances) * 0.01)])
p_thickness['pF'] = np.nanmean(distances)

In [44]:
p_thickness

{'pF.aSD': 8.33056196337216,
 'pF.aMav': 61.706876492216765,
 'pF.aMiv': 0.4071044791712436,
 'pF': 5.071628815133651}